# Proyecto
* Karen Fuentes
* Marco Ramirez
* Jenifer Arce

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
 # Instalación de librerias
import pandas as pd
import numpy as np
import sys
import re, string, unicodedata
import contractions
import inflect
import nltk
from nltk import word_tokenize, sent_tokenize, PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = nltk.corpus.stopwords.words('spanish')
wpt = nltk.WordPunctTokenizer()
ps = PorterStemmer()
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marcoramirez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marcoramirez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
def getTrainData():
    df = pd.read_csv('fake_news_spanish.csv',
                    encoding='utf-8', sep=";")
    return df

def getTestData():
    data_test = pd.read_csv(
        'fake_news_test.csv', encoding='utf-8', sep=";")
    return data_test

In [4]:
print('data spanish', getTrainData().shape)
print('data test', getTestData().shape)

data spanish (57063, 5)
data test (1000, 4)


In [5]:
trainData = getTrainData()
trainData.dtypes

ID             object
Label           int64
Titulo         object
Descripcion    object
Fecha          object
dtype: object

In [6]:
trainData.head(10)

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018
5,ID,1,"El PNV consolida su mayoría, el PSE salva los ...",Los nacionalistas consiguen las alcaldías de B...,26/05/2019
6,ID,0,El exconsejero Núria Marín pide el indulto en ...,Sus familiares aluden a su honestidad e integr...,16/09/2022
7,ID,1,La Fiscalía pide prisión incondicional para lo...,Suprime el delito de rebelión que les imputó i...,26/09/2019
8,ID,1,"José Manuel Pérez Tornero, el creador de la te...",El futuro presidente de RTVE es licenciado en ...,25/02/2021
9,ID,0,La 'Ayusización' del BNG: Santiago Abascal ins...,Pablo Santiago Abascal planea vivir de las ren...,10/05/2021


In [7]:
getTestData().head(5)

,ID,Titulo,Descripcion,Fecha
0,ID,La mesa del congreso censura un encuentro inte...,"Portavoces de Ciudadanos, PNV, UPN, PSOE, Unid...",30/10/2018
1,ID,La brecha digital que dificulta el acceso de a...,No es la primera vez que los ciudadanos vulner...,15/03/2023
2,ID,PP apremia al EQUO a presentar una propuesta d...,El partido morado reprocha que los socialistas...,01/07/2019
3,ID,De soberano garante de la democracia a rey cor...,"La renuncia de Felipe VI a su herencia, proced...",16/03/2020
4,ID,El Gobierno aprobará este martes detraer los b...,El Ejecutivo también prorrogará la suspensión ...,13/09/2021


In [8]:
testData = getTestData()
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           1000 non-null   object
 1   Titulo       999 non-null    object
 2   Descripcion  1000 non-null   object
 3   Fecha        1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB


In [9]:
trainData = getTrainData()
print(trainData['Label'].value_counts())

print('='*10, 'datos en nan', '='*10)

print(trainData["ID"].isna().sum())
print(trainData["Label"].isna().sum())
print(trainData["Titulo"].isna().sum())
print(trainData["Descripcion"].isna().sum())
print(trainData["Fecha"].isna().sum())
trainData = trainData[trainData["Titulo"].notna()]
print('='*10, 'Distribución labels', '='*10)
print('label 1:',round(trainData[trainData['Label'] == 1].shape[0]/trainData.shape[0],2),'%')
print('label 0:',round(trainData[trainData['Label'] == 0].shape[0]/trainData.shape[0],2),'%')


Label
1    33158
0    23905
Name: count, dtype: int64
========== datos en nan ==========
0
0
16
0
0
========== Distribución labels ==========
label 1: 0.58 %
label 0: 0.42 %


In [10]:
print(trainData["ID"].isna().sum())
print(trainData["Label"].isna().sum())
print(trainData["Titulo"].isna().sum())
print(trainData["Descripcion"].isna().sum())
print(trainData["Fecha"].isna().sum())

trainData = trainData[trainData["Titulo"].notna()]

0
0
0
0
0


In [11]:
from num2words import num2words

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
          new_word = unicodedata.normalize('NFKD', word).encode(
              'ascii', 'ignore').decode('utf-8', 'ignore')
          new_words.append(new_word)
    return new_words


def normalize_text(text):
    text = text.replace('á', 'a')
    text = text.replace('é', 'e')
    text = text.replace('í', 'i')
    text = text.replace('ó', 'o')
    text = text.replace('ú', 'u')
    text = text.replace('ü', 'u')
    text = text.replace('ñ', 'n')

    # Eliminate punctuation and special characters by replacing them
    text = text.replace('(', '').replace(')', '')
    text = text.replace('[', '').replace(']', '')
    text = text.replace('{', '').replace('}', '')
    text = text.replace('<', '').replace('>', '')
    return text.lower()

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    return [normalize_text(word) for word in words]


def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words

def replace_numbers(words):
   """Replace all interger occurrences in list of tokenized words with textual representation"""
   new_words = []
   for word in words:
       if word.isdigit():
           new_word = num2words(word, lang='es')
           new_words.append(new_word)
       else:
           new_words.append(word)
   return new_words


def remove_stopwords(words):
    nuevas_palabras = []
    for palabra in words:
        if palabra is not None:
            if palabra not in stopwords.words('spanish'):
                nuevas_palabras.append(palabra)
    return nuevas_palabras


def corregir_contracciones_espanol(texto):
    texto = texto.replace('al ', 'a el ').replace(
        'al.', 'a el.')  # "al" a "a el"
    # "del" a "de el"    # Agrega más reglas aquí según sea necesario
    texto = texto.replace('del ', 'de el ').replace('del.', 'de el.')
    return texto

def preprocessing(words):
    words = words.split()
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

In [12]:
from nltk.stem.snowball import SnowballStemmer


def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = SnowballStemmer("spanish")
    return [stemmer.stem(word) for word in words]


def lemmatize_verbs(words):
    """Simple lemmatization for verbs in list of tokenized words"""
    # Simple rule-based lemmatization (you can extend this as needed)
    lemmatized_words = []
    for word in words:
        if word.endswith('ar'):
            lemmatized_words.append(word[:-2])  # Remove 'ar' (basic rule)
        elif word.endswith('er'):
            lemmatized_words.append(word[:-2])  # Remove 'er' (basic rule)
        elif word.endswith('ir'):
            lemmatized_words.append(word[:-2])  # Remove 'ir' (basic rule)
        else:
            # Return the word as is if no rule applies
            lemmatized_words.append(word)
    return lemmatized_words


def stem_and_lemmatize(words):
    """Stem and Lemmatize words"""
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

In [13]:
# trainData['Descripcion2'] = trainData['Descripcion'].apply(
#     corregir_contracciones_espanol)
# trainData['Descripcion2'] = trainData['Descripcion2'].apply(
#     preprocessing)
# trainData['Descripcion3'] = trainData['Descripcion2'].apply(
#     lambda x: ' '.join(map(str, x)))
# print(len(trainData))
# trainData.head()

In [14]:
def get_good_test_data(full_data= False):
    trainData = getTrainData()
    trainData = trainData[trainData["Titulo"].notna()]
    if full_data:
        trainData['Descripcion'] = trainData['Titulo'] + \
            trainData['Descripcion']
    trainData['Descripcion2'] = trainData['Descripcion'].apply(
        corregir_contracciones_espanol)
    trainData['Descripcion2'] = trainData['Descripcion2'].apply(
        preprocessing)
    trainData['Descripcion2'] = trainData['Descripcion2'].apply(
        stem_and_lemmatize)
    trainData['Descripcion3'] = trainData['Descripcion2'].apply(
        lambda x: ' '.join(map(str, x)))
    return trainData

In [21]:
from sklearn.naive_bayes import MultinomialNB


def superTrain(dataframe):
    X = dataframe['Descripcion3']
    y = dataframe['Label']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42)


    # Crear los modelos
    models = {
        "Logistic Regression": LogisticRegression(),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "SVM": SVC(),
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        "Naive Bayes": MultinomialNB()
    }

    # Resultados de los modelos
    results = {}

    # Entrenar y evaluar cada modelo
    for name, model in models.items():
        pipeline = make_pipeline(
            TfidfVectorizer(),  # Vectorización de texto
            model
        )

        # Entrenar el modelo
        pipeline.fit(X_train, y_train)

        # Predecir y evaluar el modelo
        y_pred = pipeline.predict(X_test)

        # Guardar los resultados del reporte de clasificación
        results[name] = classification_report(y_test, y_pred, output_dict=True)

    # Imprimir los resultados de todos los modelos
    for model_name, report in results.items():
        print(f"\nModelo: {model_name}")
        print("Clasificación detallada:")
        print(pd.DataFrame(report).T)
    return results

In [22]:
trainData = get_good_test_data(1)
goodTrain = superTrain(trainData)

In [20]:
from pprint import pprint
for keys in goodTrain:
    print(keys)
    pprint(goodTrain[keys]['macro avg']['f1-score'])

NameError: name 'goodTrain' is not defined

In [37]:
goodTrain['XGBoost']

{'0': {'precision': 0.96545601291364,
  'recall': 0.8374404928591431,
  'f1-score': 0.8969033515783159,
  'support': 7142.0},
 '1': {'precision': 0.8936813186813187,
  'recall': 0.9785420635716434,
  'f1-score': 0.9341884841813047,
  'support': 9973.0},
 'accuracy': 0.9196611159801343,
 'macro avg': {'precision': 0.9295686657974793,
  'recall': 0.9079912782153933,
  'f1-score': 0.9155459178798103,
  'support': 17115.0},
 'weighted avg': {'precision': 0.9236325232508332,
  'recall': 0.9196611159801343,
  'f1-score': 0.9186295933223771,
  'support': 17115.0}}

In [55]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import pandas as pd


def tune_models_cv(dataframe, cv_folds=5):
    X = dataframe['Descripcion3']
    y = dataframe['Label']

    param_grid_xgb = {
        'xgbclassifier__n_estimators': [200, 250, 300],
        'xgbclassifier__max_depth': [7, 9, 11],
        'xgbclassifier__learning_rate': [0.2, 0.3, 0.5]
    }

    # param_grid_rf = {
    #     'randomforestclassifier__n_estimators': [100, 200, 300],
    #     'randomforestclassifier__max_depth': [None, 10, 20],
    #     'randomforestclassifier__min_samples_split': [2, 5, 10]
    # }

    models = {
        "XGBoost": (XGBClassifier(eval_metric='logloss'), param_grid_xgb),
        # "Random Forest": (RandomForestClassifier(random_state=42), param_grid_rf)
    }

    best_models = {}

    for name, (model, param_grid) in models.items():
        pipeline = make_pipeline(TfidfVectorizer(), model)
        grid_search = GridSearchCV(
            pipeline, param_grid, cv=cv_folds, scoring='f1_macro', n_jobs=-1, verbose=1
        )
        grid_search.fit(X, y)

        best_model = grid_search.best_estimator_
        cv_scores = cross_val_score(
            best_model, X, y, cv=cv_folds, scoring='f1_macro', n_jobs=-1)

        best_models[name] = best_model
        print(f"\nMejor modelo para {name}:")
        print(grid_search.best_params_)
        print(
            f"Mejor f1-score en validación cruzada: {grid_search.best_score_:.4f}")
        print(
            f"F1-score promedio en validación cruzada: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

    return best_models

In [53]:
trainData = get_good_test_data(1)

In [57]:
# best_models = tune_models_cv(trainData)

### Mejor modelo para XGBoost:
{'xgbclassifier__learning_rate': 0.2, 'xgbclassifier__max_depth': 7, 'xgbclassifier__n_estimators': 200}

Mejor f1-score en validación cruzada: 0.9218

F1-score promedio en validación cruzada: 0.9218 (+/- 0.0022)


### Mejor modelo Random Forest
{'randomforestclassifier__max_depth': None, 'randomforestclassifier__min_samples_split': 10, 'randomforestclassifier__n_estimators': 300}

Mejor f1-score en validación cruzada: 0.8991

F1-score promedio en validación cruzada: 0.8991 (+/- 0.0019)